In [ ]:
sm = snakemake

In [ ]:
import spherpro.bro as spb
import spherpro.datastore as spd
import spherpro.library as spl
import spherpro.configuration as conf
import spherpro.db as db

import matplotlib.pyplot as plt

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
class C:  
    fn_config = sm.input.fn_config
    filters = (('is-sphere', False),
    ('is-ambiguous', True),
    ('is-small', True),
    ('is-notborder', False),
    ('is-maincomponent', False),
                   ('is-qchq', False)
)
    
OBJECT_TYPE = 'cell'

In [ ]:
class V:
    fil_value=db.object_filters.filter_value.key

In [ ]:
bro = spb.get_bro(C.fn_config)

In [ ]:
def get_filter_dat(imgid, fil_name):
    q = (bro.session.query(db.object_filters, db.objects.object_number)
         .join(db.object_filter_names)
         .filter(db.object_filter_names.object_filter_name == fil_name)
         .join(db.objects)
         .filter(db.objects.image_id == imgid)
        .filter(db.objects.object_type == OBJECT_TYPE)
        )
    dat = bro.doquery(q)
    dat[db.images.image_id.key] = imgid
    dat[db.objects.object_type.key] = OBJECT_TYPE
    return dat
    
def plot_filter(imgid, fil_name, keepval=True, ax=None, **kwargs):
    dat = get_filter_dat(imgid, fil_name)
    if not keepval:
        dat[db.object_filters.filter_value.key] = dat[db.object_filters.filter_value.key] == keepval
    img = bro.plots.heatmask.assemble_heatmap_image(dat, value_var=V.fil_value)
    ax = bro.plots.heatmask.do_heatplot(img, ax=ax, title=f'{fil_name}', **kwargs)
    return ax
    

In [ ]:

imid = 2361

In [ ]:
for imgid in (123,12,1,2,3,4,5,6,7,8, 23 ,53, 73,123,32):
    fig, axs = plt.subplots(ncols=len(C.filters), dpi=200)
    for (filname, val), ax in zip(C.filters, axs):
        dat = plot_filter(imgid, filname, keepval= (val), ax=ax,colorbar=False,)
        ax.axis('off')
    plt.suptitle(f'Imag: {imgid}')
    plt.tight_layout()
    plt.show()
    plt.close()
 

In [ ]:
plot_filter(10, 'is-sphere')

In [ ]:
img = bro.plots.heatmask.assemble_heatmap_image(dat, value_var=V.fil_value)

In [ ]:
bro.plots.heatmask.do_heatplot(img)